<a href="https://colab.research.google.com/github/elangbijak4/Quantum_Security_Research/blob/main/Demo_Protokol_Quantum_Key_Distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 14.1 MB/s eta 

In [2]:
import cirq
import numpy as np

def random_bits(n):
    return np.random.randint(2, size=n)

def random_bases(n):
    return np.random.randint(2, size=n)

def encode_message(bits, bases):
    qubits = [cirq.GridQubit(0, i) for i in range(len(bits))]
    circuit = cirq.Circuit()

    for i, (bit, base) in enumerate(zip(bits, bases)):
        if bit == 1:
            circuit.append(cirq.X(qubits[i]))
        if base == 1:
            circuit.append(cirq.H(qubits[i]))

    return qubits, circuit

def measure_message(qubits, bases):
    circuit = cirq.Circuit()

    for i, base in enumerate(bases):
        if base == 1:
            circuit.append(cirq.H(qubits[i]))
        circuit.append(cirq.measure(qubits[i], key=f'q{i}'))

    return circuit

def simulate(circuit):
    simulator = cirq.Simulator()
    result = simulator.run(circuit)
    return result

# Simulate BB84 protocol
n = 10  # Number of qubits

# Step 1: Alice creates random bits and bases
alice_bits = random_bits(n)
alice_bases = random_bases(n)

# Step 2: Alice encodes her bits
qubits, alice_circuit = encode_message(alice_bits, alice_bases)

# Step 3: Bob chooses random bases
bob_bases = random_bases(n)

# Step 4: Bob measures the received qubits
bob_circuit = measure_message(qubits, bob_bases)

# Combine Alice's and Bob's circuits
full_circuit = alice_circuit + bob_circuit
print("Circuit:")
print(full_circuit)

# Step 5: Simulate the circuit
result = simulate(full_circuit)
print("Results:")
print(result)

# Extract Bob's measurements
bob_bits = [result.measurements[f'q{i}'][0] for i in range(n)]

# Step 6: Alice and Bob compare bases and extract the key
key = [alice_bits[i] for i in range(n) if alice_bases[i] == bob_bases[i]]
print("Alice's bits:", alice_bits)
print("Alice's bases:", alice_bases)
print("Bob's bases:", bob_bases)
print("Bob's bits:", bob_bits)
print("Key:", key)

Circuit:
(0, 0): ───X───H───M('q0')─────────────

(0, 1): ───X───────H─────────M('q1')───

(0, 2): ───────────H─────────M('q2')───

(0, 3): ───X───H───H─────────M('q3')───

(0, 4): ───X───────H─────────M('q4')───

(0, 5): ───X───H───H─────────M('q5')───

(0, 6): ───X───H───M('q6')─────────────

(0, 7): ───────────M('q7')─────────────

(0, 8): ───H───────M('q8')─────────────

(0, 9): ───H───────M('q9')─────────────
Results:
q0=1
q1=0
q2=0
q3=1
q4=0
q5=1
q6=1
q7=0
q8=0
q9=1
Alice's bits: [1 1 0 1 1 1 1 0 0 0]
Alice's bases: [1 0 0 1 0 1 1 0 1 1]
Bob's bases: [0 1 1 1 1 1 0 0 0 0]
Bob's bits: [array([1], dtype=int8), array([0], dtype=int8), array([0], dtype=int8), array([1], dtype=int8), array([0], dtype=int8), array([1], dtype=int8), array([1], dtype=int8), array([0], dtype=int8), array([0], dtype=int8), array([1], dtype=int8)]
Key: [1, 1, 0]
